In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os

from imutils.video import VideoStream
import imutils

In [2]:
def detect_and_predict_mask(frame , faceNet , maskNet): #( video frame ,facedetect model,mobilenet_v2 model)
    # taking then dims of frame and construct the blob
    (h,w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0,(300,300),(104.0,177.0,123.0))# resize the image (RGB only)
    faceNet.setInput(blob)  
    detections = faceNet.forward()
    
    #initialize our list of faces , their corresponding locations and list of predictions
    faces = []
    locs =[]
    preds = []
    
    for i in range(0,detections.shape[2]):
            confidence = detections[0 ,0 ,i ,2]
        #     print("confidence: " ,confidence)
            if confidence> 0.5:
                # getting the x and y coordinates 
                box = detections[0,0,i,3:7]*np.array([w,h,w,h])
                (startx , starty , endx , endy) = box.astype('int')
                (startx , starty)=(max(0 , startx) , max(0 ,starty))
                (endx , endy) = (min(w-1, endx) , min(h-1,endy))
                #Extract the face ROI , convert it from BRG to RGB channel , resize it to (224x224) and preprocess it
                face = frame[starty:endy , startx:endx]
                face = cv2.cvtColor(face , cv2.COLOR_BGR2RGB)
                face= cv2.resize(face,(224,224))
                face = img_to_array(face)
                face = preprocess_input(face)

                faces.append(face)
                locs.append((startx , starty , endx , endy))
   
        # only make a prediction if atleast one face was detected
            if len(faces)>0:
                faces= np.array(faces , dtype ='float32')
                preds = maskNet.predict(faces , batch_size =12)
            return (locs , preds)
        
    
    
    
    


In [3]:
# Importing face detectors
protoPath = os.path.sep.join([r'C:\Users\Ritika Singh\Desktop\Corona Virus Mask Detection','deploys.prototxt'])
weightsPath = os.path.sep.join([r'C:\Users\Ritika Singh\Desktop\Corona Virus Mask Detection','res10_300x300_ssd_iter_140000.caffemodel'])
faceNet = cv2.dnn.readNet(protoPath,weightsPath)

In [4]:
maskNet = load_model(r'C:\Users\Ritika Singh\Desktop\Corona Virus Mask Detection\my_weights.h5')

In [5]:
vs = VideoStream(src = 0).start()

while True :
    # grab the frame from the threaded video stream and resize it to have maximum of 400  pixels
    frame = vs.read()
    frame = imutils.resize(frame , width = 400)
    # detect and predict
    (locs , preds) = detect_and_predict_mask(frame , faceNet , maskNet)
    # loop over the face detected face locations and their corresponding locations
    
    for (box , pred )  in zip(locs , preds):
        (startx , starty , endx , endy) = box
        (mask, without_mask) =pred
        #determine the class label and color we will use to draw the bounding box and text
        label = "Mask" if mask>without_mask else "No Mask"
        color = (0,255,0) if label == 'Mask' else (0,0,255)
        #Including the probability in label

        label ="{} : {:.2f}%".format(label , max(mask ,without_mask)*100)
        
        # Display the label and bounding boxes
        
        cv2.putText(frame, label ,(startx ,starty-10) , cv2.FONT_HERSHEY_SIMPLEX , 0.45 , color,2)# diplay the text below
        cv2.rectangle(frame ,(startx ,starty),(endx, endy) , color , 2)
    
     # show the output frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1)& 0xFF #wait for some milliseconds
    if key == ord('q'):
        break
cv2.destroyAllWindows()
vs.stop()